In [58]:
import numpy as np
import pandas as pd

In [59]:
movies= pd.read_csv('data/tmdb_5000_movies.csv')
credits= pd.read_csv('data/tmdb_5000_credits.csv')

In [60]:
print(movies.shape, credits.shape)

(4803, 20) (4803, 4)


In [61]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [62]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [63]:
movies=movies.merge(credits, on='title')

In [64]:
movies.shape

(4809, 23)

In [65]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## Data Preprocessing

In [66]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [67]:
column_list=['budget','original_language','homepage','original_title','production_companies','production_countries', 'revenue','spoken_languages', 'status','tagline', 'vote_average',
       'vote_count', 'id','tagline','release_date','runtime','popularity' ]
movies.drop(columns=column_list, inplace=True)

In [68]:
movies.shape

(4809, 7)

In [69]:
movies.head(1)

,genres,keywords,overview,title,movie_id,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [70]:
movies.isnull().sum()

genres      0
keywords    0
overview    3
title       0
movie_id    0
cast        0
crew        0
dtype: int64

In [71]:
movies.dropna(inplace=True, axis=0)

In [72]:
movies.isnull().sum()

genres      0
keywords    0
overview    0
title       0
movie_id    0
cast        0
crew        0
dtype: int64

In [73]:
movies.duplicated().sum()

np.int64(0)

In [74]:
movies.shape

(4806, 7)

In [75]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [76]:
#columns like genre and keywords are strings and need to be converted to lists for preprocessing
import ast
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
       l.append(i['name'])
    return l

In [77]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [78]:
movies.head()

,genres,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [79]:
movies.iloc[0]['crew']

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [80]:
#Taking only the top 5 members from the cast
def convert_cast(obj):
    l=[]
    count=0
    for i in ast.literal_eval(obj):
        if count<5:
            l.append(i['name'])
            count+=1
        else:
            break
    return l
        

In [81]:
#We only require director's name from the crew data
def convert_crew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
        else:
            continue
    return l
        

In [82]:
movies['cast']=movies['cast'].apply(convert_cast)
movies['crew']=movies['crew'].apply(convert_crew)

In [83]:
movies['overview']=movies['overview'].apply(lambda x : x.split())

In [84]:
movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])


In [85]:
movies.head()

,genres,keywords,overview,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [86]:
#Creating a tags columns which contains all the relevant information about the movie
movies['tags']=movies['overview']+ movies['genres']+ movies['keywords']+movies['cast']+ movies['crew']

In [87]:
movies.head()

,genres,keywords,overview,title,movie_id,cast,crew,tags
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [88]:
movies.drop(columns=['overview','genres', 'keywords', 'cast', 'crew'], inplace=True )

In [89]:
movies.head()

,title,movie_id,tags
0,Avatar,19995,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,Pirates of the Caribbean: At World's End,285,"[Captain, Barbossa,, long, believed, to, be, d..."
2,Spectre,206647,"[A, cryptic, message, from, Bond’s, past, send..."
3,The Dark Knight Rises,49026,"[Following, the, death, of, District, Attorney..."
4,John Carter,49529,"[John, Carter, is, a, war-weary,, former, mili..."


In [90]:
#converting the list of words to strings
movies['tags']=movies['tags'].apply(lambda x: " ".join(x))
movies['tags']= movies['tags'].apply(lambda x:x.lower())

In [91]:
#applying steming to the data so that words like activity and activities are treated as same

#[activity, activities]
#[activity, activity]

In [92]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [93]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)   #returning a stemmed string

In [94]:
movies['tags']=movies['tags'].apply(stem)

In [95]:
movies.head()

,title,movie_id,tags
0,Avatar,19995,"in the 22nd century, a parapleg marin is dispa..."
1,Pirates of the Caribbean: At World's End,285,"captain barbossa, long believ to be dead, ha c..."
2,Spectre,206647,a cryptic messag from bond’ past send him on a...
3,The Dark Knight Rises,49026,follow the death of district attorney harvey d...
4,John Carter,49529,"john carter is a war-weary, former militari ca..."


## Vectorisation

In [96]:
from sklearn.feature_extraction.text import CountVectorizer
#consider the 50000 most used words and exclude stop words
cv= CountVectorizer(max_features= 5000, stop_words='english')

In [97]:
vectors=cv.fit_transform(movies['tags']).toarray()  #converting a sparse matrix to numpy array

In [98]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [99]:
vectors.shape   

(4806, 5000)

In [100]:
print(cv.get_feature_names_out())  #printing the most common words

['000' '007' '10' ... 'zone' 'zoo' 'zooeydeschanel']


## Calculating cosine similarity between vectors

using cosine similarity and distance since euclidean distance is not a good measure in higher dimensionality spaces

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

In [102]:
#calculating the similarity between each movie
similarity = cosine_similarity(vectors)
print(similarity[0])

[1.         0.08111071 0.08471737 ... 0.04543109 0.         0.        ]


In [103]:
similarity.shape

(4806, 4806)

In [104]:
#sorting based on the distances, enumerate helps in keeping the index position
sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x:x[1])[1:6]

[(1214, np.float64(0.28878211074619464)),
 (507, np.float64(0.257841025556124)),
 (3728, np.float64(0.25391668753850405)),
 (582, np.float64(0.24511108480187255)),
 (539, np.float64(0.2433321316961438))]

## Function to Recommend Movies

In [105]:
def recommend(movie):
    movie_index= movies[movies['title']==movie].index[0]
    distances= similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:8]
    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [106]:
recommend('Snow White and the Huntsman')    #testing a random movie

Snow White and the Seven Dwarfs
Mirror Mirror
The Snow Queen
The Huntsman: Winter's War
Queen of the Damned
One Night with the King
Alice in Wonderland


## Saving the Data

In [110]:
import joblib

joblib.dump(movies.to_dict(), 'models/movies.joblib')
joblib.dump(similarity, 'models/similarity.joblib')


['models/similarity.joblib']

# Starting the Streamlit App (fronend)

In [ ]:
!streamlit run app.py

to stop the frontend: `stop the above codeblock`

> Note to Devs: the above block should be last, as it is meant for local deployment and not other purposes